# Evolving sorting networks through coevolution

In this lab session we are going to leverage coevolution to evolve **sorting networks**. Sorting networks are algorithms designed to arrange a sequence of elements into a specific order, usually ascending or descending. The key feature of sorting networks is that they use a fixed set of comparisons and swaps to achieve the sorting, and this set of operations is independent of the input data ([wikipedia](https://en.wikipedia.org/wiki/Sorting_network)).

We will represent networks as lists of comparators, where each comparator is a pair of indices indicating which elements to compare.

First of all, we import the random module and set the seed.

In [1]:
import random
random.seed(0)

Let us define a function which sorts an array with a given sorting network.

In [2]:
def eval_sorting_network(sn, array):
    """Evaluate the sorting network on the given array."""
    for i in range(len(sn)):
        a, b = sn[i]
        if array[a] > array[b]:
            array[a], array[b] = array[b], array[a]    
    return array

In [4]:
sn=[[0,2], [1,3], [0,3] ,[1,2], [0,1], [2,3]]
eval_sorting_network(sn, [6,4,2,5])

[2, 4, 5, 6]

Now, we write the code to initialize the 2 competing populations: one for the networks and one for the arrays.

In [5]:
def get_random_network(value_range, depth):
    return [sorted(random.sample(value_range, k=2)) for _ in range(0, depth)]

In [6]:
def init_array_population(value_range, dim, pop_size):
    return [random.sample(value_range, k=dim) for _ in range(0, pop_size)]

def init_network_population(value_range, min_depth, max_depth, pop_size):
    return [get_random_network(value_range, random.randint(min_depth, max_depth)) for _ in range(0, pop_size)]

Let us define the 2 fitness functions. A network will have good fitness if it can sort many arrays, while an array will have good fitness if it can 'trick' many networks.

In [7]:
def net_fitness(net_pop, arr_pop, k=10):
    net_scores = [0]*len(net_pop)

    for i in range(len(net_pop)):
        for j in range(len(arr_pop)):
            arr = arr_pop[j].copy()
            eval_sorting_network(net_pop[i], arr)
            net_scores[i] += k - sum([1 for x, y in zip(arr, sorted(arr)) if x != y])
    
    return net_scores

def arr_fitness(net_pop, arr_pop, k=10):
    arr_scores = [0]*len(arr_pop)

    for i in range(len(arr_pop)):
        for j in range(len(net_pop)):
            arr = arr_pop[i].copy()
            eval_sorting_network(net_pop[j], arr)
            arr_scores[i] += k - sum([1 for x, y in zip(arr, sorted(arr)) if x != y])
    return arr_scores

We now implement the tournament selection and the crossover. We can use the same functions for both the populations.

In [8]:
def tournament_selection(pop, scores, k):
  tournament = random.choices(range(len(pop)), k=k)
  return pop[tournament[scores.index(max([scores[i] for i in tournament]))]]


In [9]:
def one_point_crossover(x, y):
    """Perform one point crossover on the two given parents."""
    pos = random.randint(0, len(x)-1)
    return x[:pos] + y[pos:]#, y[:pos] + x[pos:]

def uniform_crossover(x, y):
    """Perform uniform crossover on the two given parents."""
    return [random.choice([x[i], y[i]]) for i in range(len(x))]



We can now implement a mutation function for the arrays.

In [10]:
def array_mutation(x, value_range, p_m):
  def mutate(v):
    if random.random() < p_m:
        res = random.choice(value_range)
        while res == v: # we avoid sampling the same value
            res = random.choice(value_range)
        return res
    else:
      return v
  return [mutate(v) for v in x]

Now, we can choose one or more mutation operators for the sorting networks. You can use different ones during the evolution.

In [11]:
def net_mutation(sn, value_range, p_m):
    def mutate(v):
        if random.random() < p_m:
            res = random.choice(value_range)
            while res == v:
                res = random.choice(value_range)
            return res
        else:
            return v
    return [mutate(v) for v in sn]

We have now all the elements to write the code for a generation.

In [12]:
def get_best(pop, scores):
  return max(list(zip(scores, pop)))

In [13]:
def generation(net_pop, arr_pop, net_scores, arr_scores, crossover, arr_dim, value_range, p_m, t_size):
  pop_size = len(net_pop)
  # perform selection for both the populations

  new_net_pop = []
  new_arr_pop = []

  for _ in range(pop_size):
    new_net_pop.append(tournament_selection(net_pop, net_scores, t_size))
    new_arr_pop.append(tournament_selection(arr_pop, arr_scores, t_size))
  # perform crossover

  new_net_pop = [crossover(new_net_pop[i], new_net_pop[(i+1)%pop_size]) for i in range(pop_size)]
  new_arr_pop = [crossover(new_arr_pop[i], new_arr_pop[(i+1)%pop_size]) for i in range(pop_size)]
  # perform mutation

  new_net_pop = [net_mutation(new_net_pop[i], value_range, p_m) for i in range(pop_size)]
  new_arr_pop = [array_mutation(new_arr_pop[i], value_range, p_m) for i in range(pop_size)]

  return new_net_pop, new_arr_pop
  


We can now define our `coevolution` function.

In [14]:
def coevolution(value_range, 
                pop_size,
                arr_dim,
                min_depth,
                max_depth,
                net_fit,
                arr_fit,
                crossover,
                t_size = 10, 
                n_gen = 200,
                k_fit = 50):
  
  p_m = 1/arr_dim
  # initialize the population
  Pt = init_network_population(value_range, min_depth, max_depth, pop_size)
  Qt = init_array_population(value_range, arr_dim, pop_size)
  # evaluate the fitness
  net_scores = net_fit(Pt, Qt, k_fit)
  arr_scores = arr_fit(Pt, Qt, k_fit)
  net_history = [get_best(Pt, net_scores)[0]]
  arr_history = [get_best(Qt, arr_scores)[0]]
  
  for _ in range(0, n_gen):
    Ptm1 = Pt
    Qtm1 = Qt
    Pt, Qt = generation(Ptm1, Qtm1, net_scores, arr_scores, crossover, arr_dim, value_range, p_m, t_size)
    # evaluate the fitness
    net_scores = net_fit(Pt, Qt, k_fit)
    arr_scores = arr_fit(Pt, Qt, k_fit)
    net_history.append(get_best(Pt, net_scores)[0])
    arr_history.append(get_best(Qt, arr_scores)[0])
    
  return get_best(Pt, net_scores)[1], get_best(Qt, arr_scores)[1], net_history, arr_history

Try your code for different array dimensions and parameters.

In [15]:
best_net, best_arr, net_history, arr_history = coevolution(
    value_range=range(50), 
    pop_size=500,
    arr_dim=10,
    min_depth=15,
    max_depth=50,
    net_fit=net_fitness,
    arr_fit=arr_fitness,
    crossover=one_point_crossover,
    t_size = 10,
    n_gen = 200,
    k_fit = 50
    )

IndexError: list index out of range

In [ ]:
print(best_net)
print(best_arr)

In [ ]:
random_arr = random.sample(range(50), k = 10)
print(random_arr)
eval_sorting_network(best_net, random_arr)

In [ ]:
eval_sorting_network(best_net, best_arr)

Plot the evolution of the fitness score of the best individual for both populations

In [1141]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(net_history)
plt.ylabel('Network fitness')
plt.xlabel('Generation')
plt.show()

In [ ]:
plt.plot(arr_history)
plt.ylabel('Array fitness')
plt.xlabel('Generation')
plt.show()